In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import cv2
import os
from glob import glob
from tqdm import tqdm
from skimage.feature import hog
from skimage import exposure
import pickle

### Load training set label

In [2]:
df = pd.read_csv('./train_label.csv')
df.head()

,name,height,left,top,width,label
0,1.png,219,246,77,81,1
1,1.png,219,323,81,96,9
2,2.png,32,77,29,23,2
3,2.png,32,98,25,26,3
4,3.png,15,17,5,8,2


In [3]:
len(df)

73257

### Find and drop all row with left value < 0

In [4]:
df.drop(np.where(df.left < 0)[0], inplace=True)

In [5]:
len(df)

73228

### Create negative dataset

In [6]:
train_path = '../data/train/'

In [7]:
def pre_process(path, name, factor=1.3):
    """
    Preprocess image: RGB image --> Gray image --> Blur image (reduce noise) --> High Pass Filter (sharpen)         --> Constrast Limited Adaptive Histogram Equalization --> Exposure
    @INPUT:
        - path: image path
        - name: image name
        - factor: exposure factor
    @OUTPUT:
        - imgAHE: preprocessed image
    """
    kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]], np.float32)
    
    img = cv2.imread(os.path.join(path, name))

    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to gray image

    imgBlur = cv2.GaussianBlur(imgGray, (3,3), 0)   # reduce noise

    imgH = cv2.filter2D(imgBlur, -1, kernel)        # sharpen

    AHE = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))# Contrast Limited Adaptive Histogram Equalization 
    imgAHE = AHE.apply(imgH)
    imgAHE = np.array(imgAHE)

    img_processed = imgAHE*factor   # exposure

    return img_processed

In [8]:
def get_background_hog(im, top, bottom, left, right, winSize=(16,32), nsamples=15,
                   orientations=9, pixels_per_cell=(4,4), cells_per_block=(2,2)):
    """
    Calculate HOG feature for background patch in an image. 
    Background HOG is calculated from a patch with size winSize cut from region (top, bottom, left, right) of image.
    @INPUT:
        - im: Image need to find background HOG feature
        - top: top position to find background
        - bottom: bottom position to find background
        - left: left position to find background
        - right: right position to find background
        - winSize: background patch size: winSize[0] - height, winSize[1] - width
        - nsamples: number of background patch need to find
        - orientations: number of histogram bins in HOG
        - pixels_per_cell: HOG cell size
        - cells_per_block: number of cells in a block
    @OUTPUT:
        - bg: Array of size (nsamples * number of HOG features). Each row of bg contains HOG features 
        of a background patch
    """
    # Get image shape
    h, w = im.shape
    
    # Init background stack
    bg = np.array([])
    
    # Calculate step size
    stepc = np.int(winSize[0]/2)
    stepr = np.int(winSize[1]/2)
    
    # Find search margin
    t = top if top >= 0 else 0
    l = left if left >= 0 else 0
    b = bottom if bottom <= h-1 else h-1
    r = right if right <= w-1 else w-1
#     print(top, bottom, left, right)
    
    # Find all background patch
    cIdx = l
    while cIdx + winSize[0] <= r:
        rIdx = t
        while rIdx + winSize[1] <= b:
            # Get background patch
            patch = im[rIdx:rIdx+winSize[1], cIdx:cIdx+winSize[0]]
            
            # Calculate HOG for background patch
            fd = hog(patch, orientations=orientations, pixels_per_cell=pixels_per_cell,
                    cells_per_block=cells_per_block)
            fd = fd.reshape(1,-1)
            
            # Stack
            if bg.size:
                bg = np.vstack((bg, fd))
            else:
                bg = fd
            
            # Check if get enough background patch
            if bg.shape[0] >= nsamples:
                return bg
                
            # Change to next patch
            rIdx += stepr
        cIdx += stepc
    return bg

In [9]:
def create_negative(path, name, bbox, margin=10, winSize=(16,32), nsamples=10,
                   orientations=9, pixels_per_cell=(4,4), cells_per_block=(2,2)):
    """
    Create an negative dataset from dataset by extracting image background
    @INPUT:
        - path: path to dataset
        - name: image name need to extract background
        - bbox: Pandas dataframe, contain bounding box of numbers in image
        - margin: number of pixels away from bounding box position to extract background
        - winSize: background patch size: winSize[0] - height, winSize[1] - width
        - nsamples: number of background patch need to find
        - orientations: number of histogram bins in HOG
        - pixels_per_cell: HOG cell size
        - cells_per_block: number of cells in a block
    @OUTPUT:
        - bg: Array of size (nsamples * number of HOG features). Each row of bg contains HOG features 
        of a background patch
    """
    # Read image
    im = pre_process(path, name)
    
    # Get image size
    h, w = im.shape
    
    # Get bounding box of number in image
    top = np.min(bbox.top)
    bottom = np.max(bbox.top + bbox.height)
    left = np.min(bbox.left)
    right = np.max(bbox.left + bbox.width)
#     print(top, bottom, left, right)
    
    # Initialize output
    bg = np.array([])

    # Find background patch on the left
    bg_left = get_background_hog(im, 0, h-1, 0, left-margin, winSize, nsamples,
                                orientations, pixels_per_cell, cells_per_block)
    if bg_left.size:
        bg = bg_left
    
    if bg.shape[0] >= nsamples:
        return bg
    
    # Find background patch on the right
    bg_right = get_background_hog(im, 0, h-1, right + margin, w-1, winSize, nsamples,
                                 orientations, pixels_per_cell, cells_per_block)
    if bg_right.size:
        if bg.size:
            bg = np.vstack((bg, bg_right))
        else:
            bg = bg_right
    
    if bg.shape[0] >= nsamples:
        return bg
    
    # Find background patch on top
    bg_top = get_background_hog(im, 0, top - margin, left - margin, right + margin, winSize, nsamples,
                               orientations, pixels_per_cell, cells_per_block)
    if bg_top.size:
        if bg.size:
            bg = np.vstack((bg, bg_top))
        else:
            bg = bg_top
    
    if bg.shape[0] >= nsamples:
        return bg
    
    # Find background patch on bottom
    bg_bottom = get_background_hog(im, bottom + margin, h-1, left - margin, right + margin, winSize, nsamples,
                                  orientations, pixels_per_cell, cells_per_block)
    if bg_bottom.size:
        if bg.size:
            bg = np.vstack((bg, bg_bottom))
        else:
            bg = bg_bottom
        
    return bg

In [10]:
# Get all image in training set
names = os.listdir('../data/train/')

# Parameters
margin = 10
winSize = (16,32)
nsamples = 3
orientations = 9
pixels_per_cell = (4,4)
cells_per_block = (2,2)

# Get background HOG of all image in training set
bg = np.array([])
for name in tqdm(names):
    img_bg = create_negative(train_path, name, df[df.name == name], margin, winSize, nsamples,
                            orientations, pixels_per_cell, cells_per_block)
    if img_bg.size:
        if bg.size:
            bg = np.vstack((bg, img_bg))
        else:
            bg = img_bg

100%|████████████████████████████████████████████████████████████████████████████| 33402/33402 [41:15<00:00, 13.49it/s]


In [11]:
bg.shape

(74980, 540)

In [14]:
# Save negative data set
import pickle
filename = 'negative_set.p'
fileObj = open('./16x32/' + filename, 'wb')
pickle.dump(bg, fileObj)
fileObj.close()